In [22]:
import data_analysis.data_processing as dp
import data_analysis.data_learning as dl

# reload module to bypass caching
import importlib
importlib.reload(dp)
importlib.reload(dl)

# treat it as a long path to avoid path length issues
test_window_path_home = dp.Path(r'\\?\C:\Users\jannis\Documents\HSP_IDS\Material\Aktuell\2025-02-17_11-14-33_192.168.1.0-normal_1\1554220324.748197-1554220354.748197')
test_window_path_remote = dp.Path(r'\\?\...')

test_window_home = dp.ThirtySecWindow(test_window_path_home)
#test_window_remote = da.ThirtySecWindow(test_window_path_remote)
#dp.get_connection_features(test_window_home, True)
print(dl.create_feature_vectors(test_window_home))

#da.connection_kmeans(test_window_home)
#dl.create_feature_vectors(test_window_home)


ValueError: n_components=5 must be between 0 and min(n_samples, n_features)=4 with svd_solver='full'